In [ ]:
#Importar e instalar
import pandas as pd
import numpy as np
import pylab as plt   
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from scipy.interpolate import interp1d
from sklearn.impute import SimpleImputer
from statsmodels.tsa.arima.model import ARIMA
import warnings
#%pip install statsmodels
warnings.filterwarnings('ignore')   # para quitar esos prints

In [49]:
data = pd.read_csv('data/Migrated Data.csv', sep=',', encoding='utf-8')
#dataglob = pd.read_csv('data/fusion_GLOBAL_DATAFLOW_UNICEF_1.0_all.csv', sep=',', encoding='utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [ ]:
data.info()

In [ ]:
# Eliminar las columnas con valores nulos
df = data.dropna(axis=1)

In [ ]:
# Verificar los resultados
df.head()

In [ ]:
# Eliminar las filas que contienen "total" en la columna "AGE"
df = df[~df['AGE:Current age'].str.contains('_T: Total', case=False)]

In [ ]:
# Eliminar la columna "DATAFLOW"
df = df.drop("DATAFLOW", axis=1)

In [ ]:
# Contar la frecuencia de cada valor en la columna 'UNIT_MEASURE:Unit of measure'
unit_measure_counts = df['UNIT_MEASURE:Unit of measure'].value_counts()
print(unit_measure_counts)

In [ ]:
# Filtrar el DataFrame original manteniendo solo las filas con 'UNIT_MEASURE:Unit of measure' igual a 'PS: Persons'
df = df[df['UNIT_MEASURE:Unit of measure'] == 'PS: Persons']

# Mostrar el DataFrame actualizado
df.head()

In [ ]:
df = df.drop('SOURCE_LINK:Citation of or link to the data source',axis=1)

In [ ]:
df = df.drop('DATA_SOURCE:Data Source', axis=1)

In [ ]:
df = df.drop('OBS_FOOTNOTE:Observation footnote', axis=1)

In [ ]:
df = df.drop('UNIT_MEASURE:Unit of measure', axis=1)

In [ ]:
df.head()

In [ ]:
# Utilizar la función pivot para convertir la columna "time_period" en columnas separadas
data_pivot = df.pivot(index=['REF_AREA:Geographic area', 'INDICATOR:Indicator', 'AGE:Current age', 'STAT_POP:Statistical Population'],
                        columns='TIME_PERIOD:Time period',
                        values='OBS_VALUE:Observation Value')

# Restablecer el índice del DataFrame resultante
df2 = data_pivot.reset_index()

# Imprimir el resultado
df2

In [ ]:
df2.columns

In [ ]:
df2.to_csv('predict_migration.csv', index=False)

In [ ]:
# Cargar el DataFrame pivotado
df3 = pd.read_csv('predict_migration.csv')  # archivo CSV
df3


In [ ]:
# Paso 1: Limpiar los encabezados
df3.columns = df3.columns.str.split(':').str[-1].str.strip()

# Verificar las columnas presentes en el DataFrame
print(df3.columns)

# Paso 2: Asignar nuevos nombres a las columnas de años
columnas_anio = list(range(1990, 2022))
nuevas_columnas = ['REF_AREA', 'INDICATOR', 'AGE', 'STAT_POP'] + columnas_anio[:len(df3.columns) - 4]

# Verificar si las columnas en nuevas_columnas existen en el DataFrame
print(set(nuevas_columnas).issubset(df3.columns))
